In [16]:
import pandas as pd

file_path = '/Users/jorgemartinez/Documents/NYDSA #3 Machine Learning Project/Machine Learning Project Proposal/Ames_HousePrice.csv'
df = pd.read_csv(file_path)

In [17]:
# First, let's drop columns that are unnamed or have no meaning, and also drop PID
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Identify unnamed columns and PID
unnamed_cols = [col for col in df.columns if 'unnamed' in col.lower() or 'no meaning' in col.lower()]
columns_to_drop = unnamed_cols + ['PID']

# Drop identified columns
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')

print(f"Dropped {len(unnamed_cols)} unnamed columns and PID")
print(f"DataFrame shape after dropping columns: {df_cleaned.shape}")

Dropped 1 unnamed columns and PID
DataFrame shape after dropping columns: (2580, 80)


In [ ]:
# This is from the document Vinod gave me

# List of categorical variables where NA means something specific
meaningful_na_columns = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                        'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu',
                        'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                        'PoolQC', 'Fence', 'MiscFeature', 'MasVnrType']

# Fill NA values in these columns with appropriate labels
for col in meaningful_na_columns:
    if col in df.columns:
        df[col] = df[col].fillna('None')  # Replace NA with 'None'

In [19]:
# Select numerical columns
numerical_df = df.select_dtypes(include='number')

# Drop numerical columns with any missing values
numerical_df_no_na = numerical_df.dropna(axis=1)

# Select categorical columns
categorical_df = df.select_dtypes(include='object')

# We're NOT dropping categorical columns with missing values this time

In [20]:
from sklearn.preprocessing import OneHotEncoder

def drop_least_frequent_category(df_cat):
    new_df = pd.DataFrame()
    for col in df_cat.columns:
        counts = df_cat[col].value_counts()
        # Get the category with the fewest instances
        drop_cat = counts.idxmin()

        # Perform one-hot encoding and drop the least frequent category
        dummies = pd.get_dummies(df_cat[col], prefix=col)
        dummies = dummies.drop(f"{col}_{drop_cat}", axis=1)

        new_df = pd.concat([new_df, dummies], axis=1)
    return new_df

In [21]:
# Apply the custom one-hot encoding
categorical_encoded = drop_least_frequent_category(categorical_df)

print("Shape after encoding categorical variables:", categorical_encoded.shape)

Shape after encoding categorical variables: (2580, 231)


In [22]:
# Drop the target column from numerical_df if it's included
X = pd.concat([numerical_df_no_na.drop(columns=['SalePrice'], errors='ignore'),
               categorical_encoded], axis=1)

# Target variable
y = df['SalePrice']

print("Final shape of X:", X.shape)

Final shape of X: (2580, 258)


In [23]:
# Import necessary libraries
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import numpy as np
import matplotlib.pyplot as plt

# Set up 5-fold cross-validation
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

In [24]:
# Function for cross-validation
def evaluate_model_cv(model, X, y):
    # Get R² scores
    r2_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    
    # Get RMSE scores
    mse_scores = -cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(mse_scores)
    
    # Print results
    print(f"Model: {model.__class__.__name__}")
    print(f"Average RMSE: ${rmse_scores.mean():,.2f}")
    print(f"Average R²: {r2_scores.mean():.4f}")
    print("-" * 30)
    
    return {
        'model_name': model.__class__.__name__,
        'rmse_mean': rmse_scores.mean(),
        'r2_mean': r2_scores.mean()
    }

In [25]:
# 1. Multiple Linear Regression with cross-validation
print("Multiple Linear Regression:")
mlr = LinearRegression()
mlr_results = evaluate_model_cv(mlr, X, y)

Multiple Linear Regression:
Model: LinearRegression
Average RMSE: $24,434.78
Average R²: 0.8919
------------------------------


In [26]:
# 2. Ridge Regression with cross-validation
print("Ridge Regression (alpha=1.0):")
ridge = Ridge(alpha=1.0) 
ridge_results = evaluate_model_cv(ridge, X, y)

Ridge Regression (alpha=1.0):
Model: Ridge
Average RMSE: $23,133.81
Average R²: 0.9035
------------------------------


/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.56327e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.73768e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.79861e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.73974e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/anacond

In [27]:
# 3. Lasso Regression with cross-validation
print("Lasso Regression (alpha=0.01):")
lasso = Lasso(alpha=0.01)
lasso_results = evaluate_model_cv(lasso, X, y)

Lasso Regression (alpha=0.01):


/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.375e+11, tolerance: 1.090e+09
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.919e+11, tolerance: 1.096e+09
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Model: Lasso
Average RMSE: $24,273.30
Average R²: 0.8934
------------------------------


/opt/anaconda3/envs/ml_project/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.229e+11, tolerance: 1.209e+09
  model = cd_fast.enet_coordinate_descent(


In [28]:
# all results
all_results = [mlr_results, ridge_results, lasso_results]

# comparison table
results_df = pd.DataFrame(all_results)
print("\nModel Comparison:")
print(results_df)


Model Comparison:
         model_name     rmse_mean   r2_mean
0  LinearRegression  24434.782390  0.891854
1             Ridge  23133.813059  0.903509
2             Lasso  24273.295732  0.893441
